In [1]:
# Importing libraries and establishing connection to SQL database

import numpy as np
import pandas as pd
import sqlite3 as sql
import difflib
data_path = '../zippedData'
conn = sql.connect(data_path + '/im.db')

In [2]:
# Declaration of base dataframes

# Studios and gross revenue - 3387 rows
# https://www.boxofficemojo.com/
gross_df = pd.read_csv(data_path + '/bom.movie_gross.csv')

# The movie database - https://www.themoviedb.org/?language=en-US
# Reviews - 26517 rows, release date, popularity score, votes?
reviews_df = pd.read_csv(data_path + '/tmdb.movies.csv')

# Budget and gross revenues - 5782 rows
# https://www.the-numbers.com/
budget_df = pd.read_csv(data_path + '/tn.movie_budgets.csv')

# Ratings and genres by director from imdb database
director_and_ratings_df = pd.read_sql(
"""SELECT DISTINCT b.*,
            r.averagerating AS avg_rating,
            r.numvotes AS num_votes,
            p.primary_name AS director,
            p.primary_profession
FROM movie_basics b
JOIN movie_ratings r
ON b.movie_id = r.movie_id
LEFT JOIN directors d
ON b.movie_id = d.movie_id
LEFT JOIN persons p
ON d.person_id = p.person_id
""", conn)

# Ratings and genres by writers from imdb database
writers_df = pd.read_sql("""
SELECT DISTINCT b.*, r.averagerating AS avg_rating, r.numvotes AS num_votes, p.primary_name AS 'writer', p.primary_profession
FROM movie_basics b
JOIN movie_ratings r
ON b.movie_id = r.movie_id
LEFT JOIN writers w
ON b.movie_id = w.movie_id
LEFT JOIN persons p
ON w.person_id = p.person_id

""", conn)

In [3]:
# Reformatting data in budget_df

# Turned all the monetary strings into ints by replacing $ and . with empty strings
budget_df[budget_df.columns[3:]] = budget_df[budget_df.columns[3:]].replace('[\$\.]|,', '', regex=True).astype('int')

# Added column of world wide gross - production budget to get the net profit
budget_df['profit'] = budget_df['worldwide_gross'] - budget_df['production_budget']

In [4]:
# Merging budget dataframe and gross dataframe by domestic gross
# Merged on movie from budget_df and title from gross_df
budg_and_gross_df = pd.merge(budget_df, gross_df, how = "outer", left_on = 'movie', right_on = 'title')

# Dropping id and year columns, since year is represented in release_date
budg_and_gross_df.drop(['id', 'year'], axis=1, inplace=True)

# Merging gross_y and gross_x columns, preferring gross_x which seem to have higher precision
budg_and_gross_df.domestic_gross_x = budg_and_gross_df.domestic_gross_x.fillna(budg_and_gross_df.domestic_gross_y)

# Merging movie and title, preferring movie
budg_and_gross_df.movie = budg_and_gross_df.movie.fillna(budg_and_gross_df.title)

# Dropping redundant gross and title columns, dropping unnecessary studio and foreign gross columns
budg_and_gross_df.drop(['domestic_gross_y', 'title', 'studio', 'foreign_gross'], axis=1, inplace=True)

# Change dates to formatted strings
budg_and_gross_df['release_date'] = pd.to_datetime(budg_and_gross_df['release_date']).dt.strftime('%Y-%m-%d')

# New column for year
budg_and_gross_df['year'] = budg_and_gross_df['release_date'].str.split('-').str[0]

# Relabeling columns
budg_and_gross_df.columns = ['release_date', 'title', 'prod_budget', 'dom_gross', 'world_gross', 'profit', 'year']

# Reformatting money columns to integers
money_columns = ['prod_budget', 'dom_gross', 'world_gross', 'profit']
budg_and_gross_df[money_columns] = budg_and_gross_df[money_columns].astype('Int64')

# Keep only entries in budg_and_gross where prod_budget is not null
budg_and_gross_df = budg_and_gross_df.loc[budg_and_gross_df.prod_budget.notna()]

In [5]:
# Merging writers and directors dataframes on movie_id
person_df = pd.merge(writers_df, director_and_ratings_df, how='inner', on='movie_id')
person_df

,movie_id,primary_title_x,original_title_x,start_year_x,runtime_minutes_x,genres_x,avg_rating_x,num_votes_x,writer,primary_profession_x,primary_title_y,original_title_y,start_year_y,runtime_minutes_y,genres_y,avg_rating_y,num_votes_y,director,primary_profession_y
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Abrar Alvi,"writer,actor,director",Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Harnam Singh Rawail,"director,writer,producer"
1,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Gulzar,"music_department,writer,soundtrack",Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Harnam Singh Rawail,"director,writer,producer"
2,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Mahasweta Devi,writer,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Harnam Singh Rawail,"director,writer,producer"
3,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Anjana Rawail,"writer,costume_designer",Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77,Harnam Singh Rawail,"director,writer,producer"
4,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43,None,None,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43,Mani Kaul,"director,writer,actor"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182134,tt9914642,Albatross,Albatross,2017,NaN,Documentary,8.5,8,Chris Jordan,"director,writer,editor",Albatross,Albatross,2017,NaN,Documentary,8.5,8,Chris Jordan,"director,writer,editor"
182135,tt9914642,Albatross,Albatross,2017,NaN,Documentary,8.5,8,Victoria Sloan Jordan,"writer,producer",Albatross,Albatross,2017,NaN,Documentary,8.5,8,Chris Jordan,"director,writer,editor"
182136,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,None,6.6,5,Coral Cruz,"writer,miscellaneous",La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,None,6.6,5,Laura Jou,"miscellaneous,actress,director"
182137,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,None,6.6,5,Pep Puig,writer,La vida sense la Sara Amat,La vida sense la Sara Amat,2019,NaN,None,6.6,5,Laura Jou,"miscellaneous,actress,director"


In [6]:
# We want to merge these 3 tables onto each other using titles
# Standardizing the title columns of each dataframe to maximize the number of overlapping title keys
# First, replace all non alphanumeric characters with spaces, then turn any number of spaces into a single space
# Finally, strip any leading or trailing white spaces and convert all characters to lowercase
budg_and_gross_df.title = budg_and_gross_df.title.str.replace('\W', ' ', regex=True).str.replace('\s+', ' ', regex=True).str.strip().str.lower()
person_df.primary_title_x = person_df.primary_title_x.str.replace('\W', ' ', regex=True).str.replace('\s+', ' ', regex=True).str.strip().str.lower()
reviews_df.title = reviews_df.title.str.replace('\W', ' ', regex=True).str.replace('\s+', ' ', regex=True).str.strip().str.lower()

In [7]:
# Convert start year from person_df into string to match budg_and_gross_df
person_df.start_year_x = person_df.start_year_x.astype(str)

In [8]:
# Merge budg_and_gross_df and reviews_df, on titles then on release date
# to prevent movies with the same name but different dates from overwriting
# Merge left to preserve data from budg_and_gross which may not appear in reviews_df
merge_reviews = pd.merge(budg_and_gross_df, reviews_df, how='left', left_on=['title', 'release_date'] , right_on=['title', 'release_date'])

# Merge merge_reviews and person_df, on titles then on release date
# to prevent movies with the same name but different dates from overwriting
# Merge left to preserve data from merge_reviews which may not appear in person_df
merge_reviews_directors = pd.merge(merge_reviews, person_df, how='left', left_on=['title', 'year'], right_on=['primary_title_x','start_year_x'])

In [9]:
# Selecting columns to drop in order to clean up data frame
columns_to_keep = ['release_date','prod_budget','title', 'dom_gross', 'world_gross', 'profit', 'genre_ids', 'runtime_minutes_x', 'genres_x', 'writer', 'director']
columns_to_drop = [x for x in list(merge_reviews_directors.columns) if x not in columns_to_keep]

# Dropping columns based on above selection
merge_reviews_directors.drop(columns_to_drop, axis=1, inplace=True)

In [10]:
# Renaming columns
column_names = ['release_date', 'title', 'prod_budget', 'dom_gross', 'world_gross', 'profit', 'genre_ids', 'runtime_min', 'genres', 'writer', 'director']
merge_reviews_directors.columns = column_names

# Reorganizing columns
merge_reviews_directors = merge_reviews_directors.reindex(columns=['title', 'profit', 'prod_budget', 'dom_gross', 'world_gross', 'genre_ids', 'genres','director','writer','runtime_min','release_date'])

# Dropping duplicate rows
merge_reviews_directors.drop_duplicates(inplace=True)

In [11]:
# From https://www.themoviedb.org/talk/5daf6eb0ae36680011d7e6ee
# Construct list of dictionaries to decipher genre_ids column
genre_dictionary = [{"id":28,"name":"Action"},
                              {"id":12,"name":"Adventure"},
                              {"id":16,"name":"Animation"},
                              {"id":35,"name":"Comedy"},
                              {"id":80,"name":"Crime"},
                              {"id":99,"name":"Documentary"},
                              {"id":18,"name":"Drama"},
                              {"id":10751,"name":"Family"},
                              {"id":14,"name":"Fantasy"},
                              {"id":36,"name":"History"},
                              {"id":27,"name":"Horror"},
                              {"id":10402,"name":"Music"},
                              {"id":9648,"name":"Mystery"},
                              {"id":10749,"name":"Romance"},
                              {"id":878,"name":"Science Fiction"},
                              {"id":10770,"name":"TV Movie"},
                              {"id":53,"name":"Thriller"},
                              {"id":10752,"name":"War"},
                              {"id":37,"name":"Western"}]

In [12]:
# Decipher genre_ids column

# Currently, each item in column is a string, conver to list as follows:
# Strip the brackets from either end, then replace spaces with empty strings
# Then strip leading and trailing white spaces and split into list along commas
merge_reviews_directors.genre_ids = merge_reviews_directors.genre_ids.str.strip('\[\]').str.replace(" ", "").str.strip().str.split(',')

# Explode dataframe along column ids, creating duplicated rows that differ only by genre_ids value
merge_reviews_directors = merge_reviews_directors.explode('genre_ids')

# Replace empty strings or spaces with np.nan
merge_reviews_directors.genre_ids = merge_reviews_directors.genre_ids.replace(r'^\s*$', np.nan, regex=True)

# Convert entries to floats
merge_reviews_directors.genre_ids.astype('float')

# Define a function that returns 'None' if key is np.nan, otherwise, find id in dictionary and return value
def find_genre(key):
    if key is np.nan:
        return 'None'
    for dictionary in genre_dictionary:
        if dictionary['id'] == int(key):
            return dictionary['name']
    return 'None'

# Apply find_genre function to genre_ids
merge_reviews_directors.genre_ids = merge_reviews_directors.genre_ids.apply(lambda x: find_genre(x))

In [13]:
# Data was exploded due to previous conversion, now we need to collapse it back into singular titles.
# Group by movie title, then aggregate as follows:
# All numerical values by first, date by first
# Condense genre_ids, directors, and writers into lists that can be exploded later (removing duplicates)
# Reset index to return from grouby object into dataframe
merge_reviews_directors = merge_reviews_directors.groupby('title').agg({'profit': 'first',
                                              'prod_budget': 'first',
                                              'dom_gross': 'first',
                                              'world_gross': 'first',
                                              'genre_ids': lambda x: list(set(x.to_list())),
                                              'genres': 'first',
                                              'director': lambda x: list(set(x.to_list())),
                                              'writer': lambda x: list(set(x.to_list())),
                                              'runtime_min': 'first',
                                              'release_date': 'first'}).reset_index()

# Remap NaNs in genres to 'None', as consistent with genre_id row
merge_reviews_directors.genres = merge_reviews_directors.genres.fillna('None')

# Convert genres into a list rather than string
merge_reviews_directors['genres'] = merge_reviews_directors['genres'].str.split(',')

# Overwrite genres by merging genre_ids and genres columns
merge_reviews_directors.genres = merge_reviews_directors.genre_ids + merge_reviews_directors.genres

# Convert to set and back to list to remove duplicates
merge_reviews_directors.genres = merge_reviews_directors.genres.apply(lambda x: list(set(x)))

# Drop the now redundant genre_ids column
merge_reviews_directors.drop('genre_ids', axis=1, inplace=True)

In [14]:
merge_reviews_directors

,title,profit,prod_budget,dom_gross,world_gross,genres,director,writer,runtime_min,release_date
0,10 000 b c,164065678,105000000,94784201,269065678,[None],[nan],[nan],NaN,2008-03-07
1,10 cloverfield lane,103286422,5000000,72082999,108286422,"[Mystery, Thriller, Horror, Science Fiction, D...",[Dan Trachtenberg],"[Josh Campbell, Matthew Stuecken, Damien Chaze...",103.0,2016-03-11
2,10 days in a madhouse,-11985384,12000000,14616,14616,"[None, Drama]",[Timothy Hines],[Timothy Hines],111.0,2015-11-11
3,10 things i hate about you,47413950,13000000,38177966,60413950,[None],[nan],[nan],NaN,1999-03-31
4,102 dalmatians,-18058441,85000000,66941559,66941559,[None],[nan],[nan],NaN,2000-11-22
...,...,...,...,...,...,...,...,...,...,...
5693,zootopia,869429616,150000000,341268248,1019429616,"[Family, Adventure, Animation, Comedy]","[Byron Howard, Rich Moore, Jared Bush]","[Byron Howard, Jennifer Lee, Phil Johnston, Ja...",108.0,2016-03-04
5694,zulu,-14155772,16000000,0,1844228,"[Thriller, Crime, None, Drama]",[Jérôme Salle],"[Jérôme Salle, Caryl Ferey, Julien Rappeneau]",110.0,2013-12-31
5695,zwartboek,5238354,22000000,4398532,27238354,[None],[nan],[nan],NaN,2007-04-06
5696,ã l intã rieur,-2104068,3000000,0,895932,[None],[nan],[nan],NaN,2008-04-15


In [16]:
# Write to merged_data.csv for further use in other notebooks
merge_reviews_directors.to_csv('../zippedData/merged_data.csv')